# 04 — Nutritional Gap Analysis (THE HEADLINE)

**Objective:** Identify categories where existing products are nutritionally poor, creating an opening for a health-positioned private label.

## Core Insight
If most products in a category score Nutri-Score C-E, a retailer who launches an A-B product captures the health-conscious segment AND benefits from regulatory push toward mandatory labelling.

## Analyses
1. Nutritional landscape per category (Nutri-Score distributions)
2. Nutritional gap metric: `(% C/D/E) * (1 - PL penetration at A/B)`
3. Reformulation feasibility: what nutrient changes shift C/D to A/B?
4. NOVA-based ultra-processing analysis

In [ ]:
from src.analysis.nutritional_gaps import compute_nutritional_landscape, compute_nutritional_gap
from src.analysis.reformulation import analyse_reformulation
from src.visualisation.nutritional_radar import plot_nutritional_radar

In [ ]:
# TODO: Run nutritional gap analysis